In [5]:
import numpy as np
import pandas as pd
from app_service.data_service import load_data, transform_data

In [3]:
summary = load_data.get_summary_data("data/raw/summary.xlsx")
summary

,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday
Date/Time,,,,,,,,,,,,,,
2020-07-20 06:47:17.764,0:00:25.135,87,102,6,1.1724,0.0588,0,450,21,7,20,2020,30,0
2020-07-21 06:33:50.176,0:00:33.135,74,142,22,1.9189,0.1549,0,478,21,7,21,2020,30,1
2020-07-22 06:28:15.271,0:00:33.872,78,123,41,1.5769,0.3333,0,496,21,7,22,2020,30,2
2020-07-23 06:38:34.308,0:00:33.712,82,135,7,1.6463,0.0519,0,496,24,7,23,2020,30,3
2020-07-24 08:19:54.483,0:00:35.835,83,270,15,3.2530,0.0556,0,515,18,7,24,2020,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 09:34:55.009,0:21:23.488,3648,1588,245,0.4353,0.1543,1,536,45,10,1,2021,39,4
2021-10-04 07:47:17.811,0:15:51.485,3674,1978,307,0.5384,0.1552,2,449,42,10,4,2021,40,0
2021-10-06 08:41:22.988,0:18:06.223,3718,1748,333,0.4701,0.1905,4,499,49,10,6,2021,40,2


In [4]:
link_click_data = pd.read_csv("data/processed/Link Click Data.csv", parse_dates=True, index_col="Date")
link_click_data.drop(labels="Unnamed: 0", axis=1, inplace=True)
link_click_data.head()

,ArticleNumber,LinkText,Tag,Link,Clicks,Month,Day,Year,Week,Weekday
Date,,,,,,,,,,
2020-07-20,0,Celanese,3,https://en.wikipedia.org/wiki/Celanese,0.0,7,20,2020,30,0
2020-07-20,0,Clariant,4,https://en.wikipedia.org/wiki/Clariant,0.0,7,20,2020,30,0
2020-07-20,0,Orbia,5,https://en.wikipedia.org/wiki/Orbia,0.0,7,20,2020,30,0
2020-07-20,0,Westlake Chemical,6,https://en.wikipedia.org/wiki/Westlake_Chemical,1.0,7,20,2020,30,0
2020-07-20,0,by steam cracking natural gas,7,https://en.wikipedia.org/wiki/Ethylene#Industr...,0.0,7,20,2020,30,0


In [26]:
def format_date(row):
    single_digit_check = lambda number :  f"0{number}" if number < 10 else f"{number}"
    date_str = "".join(
                       [
                        single_digit_check(dt_item) for dt_item in row
                       ]
                      )
    return date_str

month_day_year = summary[["Month","Day","Year"]].values
dates = np.apply_along_axis(format_date, 1, month_day_year)

In [7]:
sample = [1, 2, 3, 4]
sample_2 = [[1,1,1] for i in sample]
new_list = []
for sub in sample_2:
    new_list += sub
new_list

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [31]:
import threading
import requests
import time

def write_article_to_file(article_date : str, headers : dict = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}):
    url_endpoint_template = "https://thecolumn.co/daily/{0}"
    html_file_path = "data/raw/articles/{0}.html".format(article_date)
    url = url_endpoint_template.format(article_date)
    url_response = requests.get(url, headers=headers)

    with open(html_file_path, "wb") as html_file:
        html_file.write(url_response.content)

start = time.perf_counter()

for url_endpoint in dates:
    write_article_to_file(url_endpoint)

end = time.perf_counter()
print(f"Code ran in {end - start} seconds")

Code ran in 60.263982921000206 seconds


In [32]:
from multiprocessing import Pool

start = time.perf_counter()
# Multithreading approach
thread_list = []

for url_endpoint in dates:
    new_thread = threading.Thread(target=write_article_to_file, args=(url_endpoint,))
    new_thread.start()
    thread_list.append(new_thread)

for t in thread_list:
    t.join()

end = time.perf_counter()
print(f"Code ran in {end - start} seconds")

Code ran in 36.67076557800101 seconds


In [37]:
from bs4 import BeautifulSoup

def get_article_content(article_date : str):
    html_file_path = "data/raw/articles/{0}.html".format(article_date)

    with open(html_file_path, "rb") as html_file:
        url_soup = BeautifulSoup(html_file)

    table_tags = url_soup.find_all("table", {"class":"container"})
    return table_tags

html_content_list = []

start = time.perf_counter()

for date in dates:
    get_article_content(date)

end = time.perf_counter()
print(f"Code ran in {end - start} seconds")

Code ran in 2.5604601340000954 seconds


In [36]:
start = time.perf_counter()

thread_list = []
for date in dates:
    new_thread = threading.Thread(target=get_article_content, args=(date,))
    new_thread.start()
    thread_list.append(new_thread)

for t in thread_list:
    t.join()

end = time.perf_counter()
print(f"Code ran in {end - start} seconds")


Code ran in 5.831153293998796 seconds
